In [1]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils
import play

pygame 2.3.0 (SDL 2.24.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from music21 import converter, instrument, note, chord, stream

In [3]:
songs = glob('Jazz/*.mid')

In [4]:
songs = songs[:3]

In [5]:
def get_notes():
    notes = []
    for file in songs:
        # converting .mid file to stream object
        midi = converter.parse(file)
        notes_to_parse = []
        try:
            # Given a single stream, partition into a part for each unique instrument
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts: # if parts has instrument parts 
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
    
        for element in notes_to_parse: 
            if isinstance(element, note.Note):
                # if element is a note, extract pitch
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                # if element is a chord, append the normal form of the 
                # chord (a list of integers) to the list of notes. 
                notes.append('.'.join(str(n) for n in element.normalOrder))
    with open('notes(1)', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

In [6]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    # Extract the unique pitches in the list of notes.
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
#    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
 #   network_output = np_utils.to_categorical(network_output)
    
 #   return (network_input, network_output)

    if len(network_output) == 0:
        # Return empty arrays if there are no sequences
        return np.array([]), np.array([])

    network_input = network_input / float(n_vocab)

    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output, num_classes=n_vocab)
    
    return network_input, network_output

In [7]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten
def create_network(network_in, n_vocab): 
    """Create the model architecture"""
    model = Sequential()
    # Set the input_shape parameter for the LSTM layer
    model.add(LSTM(128, input_shape=(network_in.shape[1], network_in.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [8]:
from keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'weights.best.music3.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

def train_network():
    """
    Get notes
    Generates input and output sequences
    Creates a model 
    Trains the model for the given epochs
    """
    
    epochs = 200
    
    notes = get_notes()
    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated')
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    #return model
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')
    return model

In [9]:
def train_network():
    """
    Get notes
    Generates input and output sequences
    Creates a model 
    Trains the model for the given epochs
    """
    
    epochs = 200
    
    notes = get_notes()
    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated')
    
    network_input, network_output = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    # Convert network_input to numpy array
    network_input = np.array(network_input)
    
    model = create_network(network_input, n_vocab)
    print('Model created')
    
    print('Training in progress')
    train(model, network_input, network_output, epochs)
    print('Training completed')


In [10]:
### Train the model 
train_network()

MidiException: badly formatted midi bytes, got: b'\x80\x04]\x94.'